# Materials associated with the paper: 

Cao, W., Williams, S., Flament, N., Zahirovic, S., Scotese, C., and Müller, R. D., 2018. Paleolatitudinal distribution of lithologic indicators of climate in a paleogeographic framework. Geological Magazine, 1-24. doi:10.1017/S0016756818000110.

### This Jupyter notebook is used to (1) reconstruct lithologic data; (2) remove sampling bias; (3) save results using different binning size: 10, 5 and 2 degree

The code in this notebook is written in Python 2.7. It utilizes standard scientific Python modules as well as modules from the open source pygplates which provides a Python API to the open source GPlates software (https://www.gplates.org/). Here, the source code has been modified to Python 3.7 by Behnam Sadeghi.


## Import Python libraries

In [1]:
import pygplates
import os
os.environ['PROJ_LIB'] = r'C:\ProgramData\Anaconda3\pkgs\proj4-5.1.0-hfa6e2cd_1\Library\share'
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_toolkits.basemap import Basemap

#%matplotlib inline

## Read data and models

In [2]:
# Matthews2016 modified model
point_feature_filename = '../Data/Lithologic_Data/Matthews2016_CEG_410.shp'
input_feature_collection = pygplates.FeatureCollection(point_feature_filename)

#Rotation:Global_EB_410-0Ma_GK07_Matthews_etal_PMAG_fixed_crossovers.rot
input_rotation_filename_Matthews2016PMAG_fixed = ['../Data/Tectonic_model/Global_EB_410-0Ma_GK07_Matthews_etal.rot']
rotation_model = pygplates.RotationModel(input_rotation_filename_Matthews2016PMAG_fixed)

In [3]:
def get_paleolithology(point_features,rotation_model):
    pX = []
    pY = []
    pAge = []
    for point in point_features:
        lithcode = point.get_shapefile_attribute('LithCode')
        if lithcode == 'C' and point.get_reconstruction_plate_id()!=0:
        #if lithcode == 'E' and point.get_reconstruction_plate_id()!=0:
        #if lithcode== 'T' or lithcode=='D' or lithcode=='G' and point.get_reconstruction_plate_id()!=0:
            BirthTime = np.median(point.get_valid_time())
            if BirthTime<410:
                pAge.append(BirthTime)
                point_rotation = rotation_model.get_rotation(BirthTime, point.get_reconstruction_plate_id(), anchor_plate_id=0) #obtain rotating rules
                reconstructed_point = point_rotation * point.get_geometry() # carry out reconstruction
                pX.append(reconstructed_point.to_lat_lon()[1])
                pY.append(reconstructed_point.to_lat_lon()[0])            
    return (pX,pY,pAge)

In [4]:
age_range_list = [(419,393),   # Early Devonian
           (393,388),   # Middle Devonian (Eifelian)
           (388,383),   # Middle Devonian (Givetian)
           (383,359),   # Late Devonian
           (359,331),   # Early Carboniferous (Tournaisian-Visean)
           (331,323),   # Early Carboniferous (Serpukhovian)
           (323,307),   # Late Carboniferous (Bashkirian-Moscovian)
           (307,299),   # Late Carboniferous (Kasimovian-Gzhelian)
           (299,290),   # Early Permian (Asselian-Sakmarian)
           (290,252),   # Middle-Late Permian (Artinskian-Lopingian)
           (252,247),   # Early Triassic
           (247,237),   # Middle Triassic
           (237,201),   # Late Triassic
           (201,164),   # Early and Middle Jurassic
           (164,145),   # Late Jurassic
           (145,113),   # Early Cretaceous (Berriasian-Aptian)
           (113,89.8),   # Late Cretaceous (Albian-Turonian)
           (89.8,66.0),   # Late Cretaceous (Coniacian-Maastrichtian)
           (66.0,56.0),   # Paleocene
           (56.0,47.8),  # Early Eocene (Ypresian?)
           (47.8,33.9),  # Middle and Late Eocene
           (33.9,23.0),  # Oligocene
           (23.0,5.3),    # Miocene
           (0.01,-0.01)] # present-day
#print age_range_list[5:]

i = 0
for i in np.arange(0,24,1):
    Age = np.mean(age_range_list[i])
    print (Age)

406.0
390.5
385.5
371.0
345.0
327.0
315.0
303.0
294.5
271.0
249.5
242.0
219.0
182.5
154.5
129.0
101.4
77.9
61.0
51.9
40.849999999999994
28.45
14.15
0.0


In [5]:
# reconstruct all data and get their x, y coordinates and ages
pX,pY,pAge = get_paleolithology(input_feature_collection,rotation_model)
#print len(pX)
#print pAge

for j in np.arange(0,24,1):
    n = 0
    for i in np.arange(0,len(pAge),1):
        if np.array(pAge[i]) == np.mean(age_range_list[j]):
            n = n+1
            #print (np.array(pAge[i]))
            #print (np.mean(age_range_list[j]))
    print (n)

3
4
11
26
189
99
291
268
298
334
19
21
228
478
86
244
162
160
116
168
233
180
384
1352


### Binning size: 5 degree

In [6]:
# remove the sampling bias

result_original = []
result_SamplingBiasRemoved = []
result_flipped = []
result_doubled = []

for i in np.arange(0,24,1):
    
    result = []
    result_0 = []
    result_1 = []
    result_2 = []
    result_3 = []
    
    index = np.where(np.array(pAge) == np.mean(age_range_list[i]))
    #print np.mean(age_range_list[i]),np.array(pAge)[index]
    
    xedges = np.arange(-180, 181, 5) #180, longtitude
    yedges = np.arange(-90, 91, 5) #90, latitude, specify the bin size
    data_hist = np.histogram2d(np.array(pX)[index],np.array(pY)[index], bins=(xedges, yedges))
    result_0 = np.nansum(data_hist[0], axis=0)
    result_original.append(result_0)
    #print type(result_original)
    
    # save the original data
    #df = pd.DataFrame(data_hist[0].T)
    #writer = pd.ExcelWriter('OriginalData_Glacial_%sMa.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter') # Evaporite
    #df.to_excel(writer, sheet_name='Sheet1')
    #writer.save()
    
    # remove the sampling bias
    result_1 = np.nansum(data_hist[0]/data_hist[0], axis=0)
    #print result_1
    result_SamplingBiasRemoved.append(result_1)
    
    #plt.pcolor(xedges,yedges,data_hist[0].T) #T means reverse the x, y axis
    
    # flip the data
    #result_2 = np.flipud(result_1[:18])+result_1[18:]
    #result_flipped.append(result_2) # add data on south hemisphere to north hemishpere
    #print result_flipped
    
    # double the data
    #result_3 = list(np.flipud(result_2)) + list(result_2) 
    #result_doubled.append(np.array(result_3)) # add data on south hemisphere to north hemishpere
    #print result_doubled

# save data for all time steps into a table
df = pd.DataFrame(result_original)
writer = pd.ExcelWriter('Output/ProcessingData_2/Coals_Matthews2016_Origianl_410-0Ma_BinningSize5.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()

# save sampling bias-corrected data for all time steps into a table
df = pd.DataFrame(result_SamplingBiasRemoved)
writer = pd.ExcelWriter('Output/ProcessingData_2/Coals_Matthews2016_SamplingBiasCorrected_410-0Ma_BinningSize5.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()

# save flipped data for all time steps into a table
#df = pd.DataFrame(result_flipped)
#writer = pd.ExcelWriter('Glacial_Golonka_Flipped_410-0Ma.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter')
#df.to_excel(writer, sheet_name='Sheet1')
#writer.save()

# save flipped data for all time steps into a table
#df = pd.DataFrame(result_doubled)
#writer = pd.ExcelWriter('Glacial_Golonka_Doubled_410-0Ma.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter')
#df.to_excel(writer, sheet_name='Sheet1')
#writer.save()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide


### Binning size: 10 degree

In [7]:
# remove the sampling bias
result_original = []
result_SamplingBiasRemoved = []
result_flipped = []
result_doubled = []

for i in np.arange(0,24,1):
    
    result = []
    result_0 = []
    result_1 = []
    result_2 = []
    result_3 = []
    
    index = np.where(np.array(pAge) == np.mean(age_range_list[i]))
    #print (np.mean(age_range_list[i]),np.array(pAge)[index])
    
    xedges = np.arange(-180, 181, 10) #180, longtitude
    yedges = np.arange(-90, 91, 10) #90, latitude, specify the bin size
    print (yedges)
    data_hist = np.histogram2d(np.array(pX)[index],np.array(pY)[index], bins=(xedges, yedges))
    result_0 = np.nansum(data_hist[0], axis=0)
    result_original.append(result_0)
    #print (type(result_original))
    
    # save the original data
    #df = pd.DataFrame(data_hist[0].T)
    #writer = pd.ExcelWriter('OriginalData_Glacial_%sMa.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter') # Evaporite
    #df.to_excel(writer, sheet_name='Sheet1')
    #writer.save()
    
    # remove the sampling bias
    result_1 = np.nansum(data_hist[0]/data_hist[0], axis=0)
    #print (result_1)
    result_SamplingBiasRemoved.append(result_1)
    
    #plt.pcolor(xedges,yedges,data_hist[0].T) #T means reverse the x, y axis
    
    # flip the data
    result_2 = np.flipud(result_1[:9])+result_1[9:]
    result_flipped.append(result_2) # add data on south hemisphere to north hemishpere
    #print (result_flipped)
    
    # double the data
    result_3 = list(np.flipud(result_2)) + list(result_2) 
    result_doubled.append(np.array(result_3)) # add data on south hemisphere to north hemishpere
    #print (result_doubled)

# save data for all time steps into a table
df = pd.DataFrame(result_original)
writer = pd.ExcelWriter('Output/ProcessingData_2/Coals_Matthews2016_Origianl_410-0Ma_BinningSize10.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()

# save sampling bias-corrected data for all time steps into a table
df = pd.DataFrame(result_SamplingBiasRemoved)
writer = pd.ExcelWriter('Output/ProcessingData_2/Coals_Matthews2016_SamplingBiasCorrected_410-0Ma_BinningSize10.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()

# save flipped data for all time steps into a table
#df = pd.DataFrame(result_flipped)
#writer = pd.ExcelWriter('Glacial_Flipped_410-0Ma.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter')
#df.to_excel(writer, sheet_name='Sheet1')
#writer.save()

# save flipped data for all time steps into a table
#df = pd.DataFrame(result_doubled)
#writer = pd.ExcelWriter('Glacial_Doubled_410-0Ma.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter')
#df.to_excel(writer, sheet_name='Sheet1')
#writer.save()

#result_1 = []
#result_1 = np.sum(result,axis=0) # sum up data in Y-axis


[-90 -80 -70 -60 -50 -40 -30 -20 -10   0  10  20  30  40  50  60  70  80
  90]
[-90 -80 -70 -60 -50 -40 -30 -20 -10   0  10  20  30  40  50  60  70  80
  90]
[-90 -80 -70 -60 -50 -40 -30 -20 -10   0  10  20  30  40  50  60  70  80
  90]
[-90 -80 -70 -60 -50 -40 -30 -20 -10   0  10  20  30  40  50  60  70  80
  90]
[-90 -80 -70 -60 -50 -40 -30 -20 -10   0  10  20  30  40  50  60  70  80
  90]
[-90 -80 -70 -60 -50 -40 -30 -20 -10   0  10  20  30  40  50  60  70  80
  90]
[-90 -80 -70 -60 -50 -40 -30 -20 -10   0  10  20  30  40  50  60  70  80
  90]
[-90 -80 -70 -60 -50 -40 -30 -20 -10   0  10  20  30  40  50  60  70  80
  90]
[-90 -80 -70 -60 -50 -40 -30 -20 -10   0  10  20  30  40  50  60  70  80
  90]
[-90 -80 -70 -60 -50 -40 -30 -20 -10   0  10  20  30  40  50  60  70  80
  90]
[-90 -80 -70 -60 -50 -40 -30 -20 -10   0  10  20  30  40  50  60  70  80
  90]
[-90 -80 -70 -60 -50 -40 -30 -20 -10   0  10  20  30  40  50  60  70  80
  90]
[-90 -80 -70 -60 -50 -40 -30 -20 -10   0  10  20  30

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide


### Binning size: 2 degree

In [8]:
# remove the sampling bias
result_original = []
result_SamplingBiasRemoved = []
result_flipped = []
result_doubled = []

for i in np.arange(0,24,1):
    
    result = []
    result_0 = []
    result_1 = []
    result_2 = []
    result_3 = []
    
    index = np.where(np.array(pAge) == np.mean(age_range_list[i]))
    #print np.mean(age_range_list[i]),np.array(pAge)[index]
    
    xedges = np.arange(-180, 181, 2) #180, longtitude
    yedges = np.arange(-90, 91, 2) #90, latitude, specify the bin size
    data_hist = np.histogram2d(np.array(pX)[index],np.array(pY)[index], bins=(xedges, yedges))
    result_0 = np.nansum(data_hist[0], axis=0)
    result_original.append(result_0)
    #print type(result_original)
    
    # save the original data
    #df = pd.DataFrame(data_hist[0].T)
    #writer = pd.ExcelWriter('OriginalData_Glacial_%sMa.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter') # Evaporite
    #df.to_excel(writer, sheet_name='Sheet1')
    #writer.save()
    
    # remove the sampling bias
    result_1 = np.nansum(data_hist[0]/data_hist[0], axis=0)
    #print (result_1)
    result_SamplingBiasRemoved.append(result_1)
    
    #plt.pcolor(xedges,yedges,data_hist[0].T) #T means reverse the x, y axis
    
    # flip the data
    result_2 = np.flipud(result_1[:45])+result_1[45:]
    result_flipped.append(result_2) # add data on south hemisphere to north hemishpere
    #print (result_flipped)
    
    # double the data
    result_3 = list(np.flipud(result_2)) + list(result_2) 
    result_doubled.append(np.array(result_3)) # add data on south hemisphere to north hemishpere
    #print (result_doubled)

# save data for all time steps into a table
df = pd.DataFrame(result_original)
writer = pd.ExcelWriter('Output/ProcessingData_2/Coals_Matthews2016_Origianl_410-0Ma_BinningSize2.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
#writer.save()

# save sampling bias-corrected data for all time steps into a table
df = pd.DataFrame(result_SamplingBiasRemoved)
writer = pd.ExcelWriter('Output/ProcessingData_2/Coals_Matthews2016_SamplingBiasCorrected_410-0Ma_BinningSize2.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()

# save flipped data for all time steps into a table
#df = pd.DataFrame(result_flipped)
#writer = pd.ExcelWriter('Glacial_Flipped_410-0Ma.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter')
#df.to_excel(writer, sheet_name='Sheet1')
#writer.save()

# save flipped data for all time steps into a table
#df = pd.DataFrame(result_doubled)
#writer = pd.ExcelWriter('Glacial_Doubled_410-0Ma.xlsx' % np.mean(age_range_list[i]), engine='xlsxwriter')
#df.to_excel(writer, sheet_name='Sheet1')
#writer.save()

#result_1 = []
#result_1 = np.sum(result,axis=0) # sum up data in Y-axis


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
